In [1]:
# https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

import nltk
import pandas as pd

In [2]:
data_path = "C://Users//alvin//DATA//dsc//cxc-2022//data//imdb_ds.csv"

raw_ds = pd.read_csv(data_path)
raw_ds.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
raw_ds["sentiment"].value_counts()

# perfectly balanced as all things should be

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [4]:
sentiment_dict = {
    "positive": 1,
    "negative": 0
}

raw_ds["sentiment_num"] = raw_ds["sentiment"].map(sentiment_dict)
raw_ds.head()

,review,sentiment,sentiment_num
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [5]:
import re

def cleanup(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^\w\s]', '', sentence)
    return sentence

cleanup(raw_ds["review"].iloc[1])

'a wonderful little production br br the filming technique is very unassuming very oldtimebbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece br br the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great masters of comedy and his life br br the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwells murals decorating every surface are terribly well done'

In [6]:
import numpy as np

glove_path = "C://Users//alvin//DATA//dsc//cxc-2022//NLP//glove.6B.50d.txt"
glove_dict = dict()
with open(glove_path,'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], np.float32)
        glove_dict[word] = vector

In [7]:
words, mats = glove_dict.keys(), np.stack(list(glove_dict.values()))

In [8]:
from sklearn.decomposition import PCA

GLOVE_DIM = 15
pca = PCA(n_components=GLOVE_DIM)
mats_10 = pca.fit_transform(mats)

In [10]:
glove_dict_10 = dict()
for i, word in enumerate(words):
    glove_dict_10[word] = mats_10[i]

In [11]:
def vectorize_glove(word):
    if word in glove_dict.keys():
        return glove_dict_10[word]
    else:
        return np.random.random((GLOVE_DIM))
    
print("hello: {} \n\nkjnkejnv: {}".format(vectorize_glove("hello"), vectorize_glove("kjnkejnv")))

hello: [ 1.753264    0.8783734   0.14174092 -1.4482158   0.6975822  -0.8384445
 -1.203791    0.27736428 -0.6394595   0.8745519   0.33310515 -0.77899134
 -1.2759787   0.28469568  0.22572689] 

kjnkejnv: [0.56048375 0.07032973 0.20614324 0.25095905 0.32144688 0.35822926
 0.48181473 0.3996473  0.10797892 0.0943158  0.76699358 0.62843729
 0.16971241 0.06944541 0.36329769]


In [12]:
MAX_SENT_LEN = 30

def vectorize_sentence(sentence):
    sent_vec = []
    words = list(reversed(nltk.word_tokenize(sentence)))
    for i in range(MAX_SENT_LEN):
        if i < len(words):
            sent_vec.append(vectorize_glove(words[i]))
        else:
            sent_vec.append(np.zeros(GLOVE_DIM))
            
    return np.stack(sent_vec)

vectorize_sentence(cleanup("I love maths"))

array([[ 1.47343481,  0.13152935,  0.21112117,  0.37849015,  0.1479634 ,
        -0.08398522, -0.54872286,  0.53612709,  0.93698561, -0.50580764,
        -1.48206913,  0.71703589,  0.83700436, -0.65213478, -0.0909888 ],
       [ 3.92047596,  2.16904998,  0.27685186, -1.32169294,  0.49764135,
        -0.11994789, -0.56772912,  0.33904338, -1.18955886,  0.37913424,
        -0.87195534,  0.07459655, -0.9969402 , -0.21704257,  0.18395908],
       [ 4.71272421,  3.11399579, -0.17340472, -1.25411999,  0.73432481,
        -0.6846047 , -0.0323642 ,  0.63478839, -0.22299461,  0.25886959,
        -1.11839449, -0.32946971, -0.79433852, -0.06401621,  0.71872574],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , 

In [13]:
def process_sentence(sentence):
    return vectorize_sentence(cleanup(sentence)).flatten()

In [14]:
from sklearn.model_selection import train_test_split
train_ds, test_ds = train_test_split(raw_ds, test_size=0.2, random_state=42)

x_train = train_ds["review"].apply(process_sentence)
x_train = np.stack(x_train.values)
y_train = train_ds["sentiment_num"].tolist()

In [31]:
np.sum(y_train)/len(y_train), np.sum(y_test)/len(y_test)

(0.499025, 0.5039)

In [20]:
x_test = test_ds["review"].apply(process_sentence)
x_test = np.stack(x_test.values)
y_test = test_ds["sentiment_num"].tolist()

In [32]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=50, max_depth=7)
model.fit(x_train, y_train)

[12:47:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [33]:
np.sum(model.predict(x_train) == y_train)/len(x_train)

0.9223

In [34]:
np.sum(model.predict(x_test) == y_test)/len(x_test)

0.6427